# Launch Site Locations Analysis with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In [116]:
# Importing Libraries
import folium
import pandas as pd
import numpy as np

from folium.plugins import MarkerCluster
from folium.features import DivIcon

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


#### 1. Mark all launch sites on a map

trying to add each site's location on a map using site's latitude and longitude coordinates

In [2]:
df = pd.read_csv('spacex_launch_geo.csv')
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
df = df[['Lat', 'Long', 'Launch Site', 'class']]
launch_sites_df = df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [38]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [39]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [58]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

In [59]:
for lat, lng, label in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df['Launch Site']):
    folium.Circle(
         location=[lat, lng],
         radius=100,
         color='#d35400',
         popup=folium.Popup(f'{label}', parse_html=True)
    ).add_to(site_map) 
    
    folium.Marker(
        location=[lat, lng],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{label}</b></div>'
        )
    ).add_to(site_map)

site_map

In [97]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'], )).add_to(site_map)
site_map


--> Appendices

**1. Now we could notice that almost the 4 sites are in porximity to the equator line due to the need of utilizing the rotation speed of Earth.**

**The Earth rotates on its axis, and at the equator, the rotational speed is higher than at higher latitudes.**

**2. We could also notice that all of SpaceX launch sites are in proximity to coast. This relates to multiple reasons, safety and overflight clearance  is one of those reasons. Emergency Abort Scenarios is a second reason to consider for the launch sites being close to the coast.**

#### 2. Mark the success/failed launches for each site on the map

Next, we're going to try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.


In [98]:
df.tail(10)


,Lat,Long,Launch Site,class,marker_colour
46,28.573255,-80.646895,KSC LC-39A,1,green
47,28.573255,-80.646895,KSC LC-39A,1,green
48,28.573255,-80.646895,KSC LC-39A,1,green
49,28.563197,-80.576820,CCAFS SLC-40,1,green
50,28.563197,-80.576820,CCAFS SLC-40,1,green
51,28.563197,-80.576820,CCAFS SLC-40,0,red
52,28.563197,-80.576820,CCAFS SLC-40,0,red
53,28.563197,-80.576820,CCAFS SLC-40,0,red
54,28.563197,-80.576820,CCAFS SLC-40,1,green
55,28.563197,-80.576820,CCAFS SLC-40,0,red


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [99]:
marker_cluster = MarkerCluster()

In [100]:
# Creating a new columnn to assign a different colour to each launch site.
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def get_marker_colour(row):
    return 'green' if row['class'] == 1 else 'red'

In [101]:
df['marker_colour'] = df.apply(lambda row:get_marker_colour(row), axis=1)

df.head()

,Lat,Long,Launch Site,class,marker_colour
0,28.562302,-80.577356,CCAFS LC-40,0,red
1,28.562302,-80.577356,CCAFS LC-40,0,red
2,28.562302,-80.577356,CCAFS LC-40,0,red
3,28.562302,-80.577356,CCAFS LC-40,0,red
4,28.562302,-80.577356,CCAFS LC-40,0,red


Now, for each launch result in `spacex_df` data frame, we're gonna add a `folium.Marker` to `marker_cluster`

In [102]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for i, record in df.iterrows():
    coordinates = [record['Lat'], record['Long']]
    
    marker = folium.Marker(coordinates, icon=folium.Icon(color='white', icon_color=record['marker_colour']))
    marker_cluster.add_child(marker)  

site_map